### Step 1: Initial Data Loading

In [ ]:
from utils.preprocess import *
# 1. Identify the paths to the soft file and the matrix file
cohort_dir = '/media/techt/DATA/GEO/Sickle_Cell_Anemia/GSE9877'
soft_file, matrix_file = geo_get_relevant_filepaths(cohort_dir)

# 2. Read the matrix file to obtain background information and sample characteristics data
background_prefixes = ['!Series_title', '!Series_summary', '!Series_overall_design']
clinical_prefixes = ['!Sample_geo_accession', '!Sample_characteristics_ch1']
background_info, clinical_data = get_background_and_clinical_data(matrix_file, background_prefixes, clinical_prefixes)

# 3. Obtain the sample characteristics dictionary from the clinical dataframe
sample_characteristics_dict = get_unique_values_by_row(clinical_data)

# 4. Explicitly print out all the background information and the sample characteristics dictionary
print("Background Information:")
print(background_info)
print("Sample Characteristics Dictionary:")
print(sample_characteristics_dict)


### Step 2: Dataset Analysis and Clinical Feature Extraction

In [ ]:
is_gene_available = False
trait_row = age_row = gender_row = None  # set to different values when applicable
convert_trait = convert_age = convert_gender = None  # define the functions when applicable

# Define sample_characteristics dictionary similarly as in STEP 1 for testing purpose
sample_characteristics = {
    0: ['cell type: blood outgrowth endothelial cell', 'At Risk: -', 'At Risk: +'],
    1: ['status: normal', 'Age: 11', 'Age: 15', 'Age: 8', 'Age: 14', 'Age: 17', 'Age: 12', 'Age: 10', 'Age: 4', 'Age: 13', 'Age: 9', 'Age: 19', 'Age: 18', 'Age: 16'],
    2: ['race: African American', 'race: Caucasian', 'Sex: F', 'Sex: M'],
    3: ['age: 50', 'age: 60', 'age: 53', 'age: 40', 'age: 41', 'age: 33', 'age: 23', 'age: 46', 'age: 31', 'age: 63', 'age: 57', 'age: 29', 'age: 44', 'age: 27', 'age: 55', 'age: 35', None],
    4: ['sex: F', 'sex: M', None]
}

# Determine if gene expression data is available
is_gene_available = True  # Based on the series summary mentioning gene expression profiling

# Data availability and type conversion functions

def convert_trait(value):
    try:
        return 1 if 'At Risk: +' in value else 0
    except:
        return None

def convert_age(value):
    try:
        return int(value.split(': ')[1])
    except:
        return None

def convert_gender(value):
    try:
        gender = value.split(': ')[1].lower()
        if gender == 'f':
            return 0
        elif gender == 'm':
            return 1
        else:
            return None
    except:
        return None

# Determine the keys for each variable in the sample characteristics data
for key, values in sample_characteristics.items():
    if any('At Risk:' in value for value in values):
        trait_row = key

    if any('Age:' in value for value in values):
        age_row = key

    if any('Sex:' in value for value in values):
        gender_row = key

# Save cohort information
save_cohort_info('GSE9877', './preprocessed/Sickle_Cell_Anemia/cohort_info.json', is_gene_available, trait_row is not None)

# Clinical feature extraction
if trait_row is not None:
    selected_clinical_data = geo_select_clinical_features(clinical_data, 'Sickle_Cell_Anemia', trait_row, convert_trait, age_row, convert_age, gender_row, convert_gender)
    csv_path = './preprocessed/Sickle_Cell_Anemia/trait_data/GSE9877.csv'
    selected_clinical_data.to_csv(csv_path)
    print(preview_df(selected_clinical_data))


### Step 3: Gene Data Extraction

In [ ]:
# 1. Use the get_genetic_data function from the library to get the gene_data from the matrix_file previously defined.
gene_data = get_genetic_data(matrix_file)

# 2. Print the first 20 row ids for the following step.
print(gene_data.index[:20])


### Step 4: Gene Identifier Review

In [ ]:
requires_gene_mapping = True


### Step 5: Gene Annotation (Conditional)

In [ ]:
# 1. Use the 'get_gene_annotation' function from the library to get gene annotation data from the soft file.
gene_annotation = get_gene_annotation(soft_file)

# 2. Use the 'preview_df' function from the library to preview the data and print out the results.
print("Gene annotation preview:")
print(preview_df(gene_annotation))


### Step 6: Gene Identifier Mapping

In [ ]:
# 1. Set the identifier_key and gene_symbol_key based on the preview of gene annotation
identifier_key = 'ID'
gene_symbol_key = 'Gene Symbol'

# 2. Get the dataframe storing the mapping between probe IDs and genes
gene_mapping_df = get_gene_mapping(gene_annotation, identifier_key, gene_symbol_key)

# 3. Apply the mapping and name the resulting gene expression dataframe "gene_data"
gene_data = apply_gene_mapping(gene_data, gene_mapping_df)


### Step 7: Data Normalization and Merging

In [ ]:
# 1. Normalize the obtained gene data with the 'normalize_gene_symbols_in_index' function from the library.
normalized_gene_data = normalize_gene_symbols_in_index(gene_data)
gene_csv_path = './preprocessed/Sickle_Cell_Anemia/gene_data/GSE9877.csv'
normalized_gene_data.to_csv(gene_csv_path)

# 2. Merge the clinical and genetic data with the 'geo_merge_clinical_genetic_data' function from the library.
merged_data = geo_merge_clinical_genetic_data(selected_clinical_data, normalized_gene_data)

# 3. Determine whether the trait and some demographic attributes in the data is severely biased, and remove biased attributes.
trait_biased, unbiased_merged_data = judge_and_remove_biased_features(merged_data, 'Sickle_Cell_Anemia')

# 4. Save the cohort information.
save_cohort_info('GSE9877', './preprocessed/Sickle_Cell_Anemia/cohort_info.json', True, True, trait_biased, merged_data)

if not trait_biased:
    # 5. If the trait is not severely biased, save the merged data to a csv file.
    csv_path = './preprocessed/Sickle_Cell_Anemia/GSE9877.csv'
    unbiased_merged_data.to_csv(csv_path)
